In [131]:
from scipy.special import roots_jacobi, eval_jacobi
import pandas as pd
import numpy as np
import uxarray as ux
import xarray as xr

In [447]:
grid_file = ux.open_grid('TEMPEST_ne30.g')
grid_file

<uxarray.Grid>
Original Grid Type: Exodus
Grid Dimensions:
  * n_node: 5402
  * n_face: 5400
  * n_max_face_nodes: 4
  * n_nodes_per_face: (5400,)
Grid Coordinates (Spherical):
  * node_lon: (5402,)
  * node_lat: (5402,)
Grid Coordinates (Cartesian):
  * node_x: (5402,)
  * node_y: (5402,)
  * node_z: (5402,)
Grid Connectivity Variables:
  * face_node_connectivity: (5400, 4)

In [431]:
index_array = np.array([1807, 3623, 3621, 1836, 1838])
index_array


array([1807, 3623, 3621, 1836, 1838])

In [432]:
test = grid_file.face_node_connectivity[index_array]
test

<xarray.DataArray 'face_node_connectivity' (n_face: 5, n_max_face_nodes: 4)>
array([[  72,   73, 2045, 2044],
       [  72,   71, 3743, 3742],
       [  74,   73, 3741, 3740],
       [2043, 2044, 2073, 2072],
       [2045, 2046, 2075, 2074]])
Dimensions without coordinates: n_face, n_max_face_nodes
Attributes:
    cf_role:      face_node_connectivity
    _FillValue:   -9223372036854775808
    start_index:  0

In [437]:
shared_count = np.sum(np.sum(np.array([test.values[:, :][i] == test.values[ :, :, None] for i in np.arange(test.values.shape[0])]),
       axis = -1), axis = -1)
shared_count

array([[4, 1, 1, 1, 1],
       [1, 4, 0, 0, 0],
       [1, 0, 4, 0, 0],
       [1, 0, 0, 4, 0],
       [1, 0, 0, 0, 4]])

In [413]:
i, j = np.where((shared_count == 1))
shared_corners = list(zip(i, j))

In [479]:
shared_corners

[(1, 2), (1, 4), (2, 1), (2, 3), (3, 2), (4, 1)]

In [499]:
# Insert 2 additional data_vars, one for shared edges and one for shared corners:

def find_neighbors(array_in):
    
    dict_corners = {} 
    dict_edges = {}
    
    for value in np.arange(array_in.n_node):
        index_array = np.where(array_in.face_node_connectivity == value)[0]
        node_array = array_in.face_node_connectivity[index_array]
        
        # Create matrix tracking which elements overlap across rows of node_array: 
        shared_counts = np.sum(np.sum(np.array([node_array.values[i, :] == node_array.values[:, :, None] for i in np.arange(node_array.shape[0])]),
                               axis = -1), axis = -1)
        
        # 1's denote shared corners, 2's denote shared edges:
        i, j = np.where(shared_counts == 1)
        shared_corners = list(zip(i, j))
        
        a, b = np.where(shared_counts == 2)
        shared_edges = list(zip(a, b))
            
        # Returns the indexes of node faces which share corners/edges:
        [dict_edges.setdefault(index_array[pair1[0]], set()).add(index_array[pair1[1]]) for pair1 in shared_edges]
        [dict_corners.setdefault(index_array[pair2[0]], set()).add(index_array[pair2[1]]) for pair2 in shared_corners]
        
        dict(sorted(dict_corners.items()))
        
        
    return(dict(sorted(dict_corners.items())), dict(sorted(dict_edges.items())))

In [550]:
test = find_neighbors(grid_file)

In [569]:
spherical_coords.isel(n_face = list(test[0][0]))

<xarray.Dataset>
Dimensions:     (n_face: 3, GLL_points: 16)
Coordinates:
  * n_face      (n_face) int64 2759 31 4471
  * GLL_points  (GLL_points) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15
Data variables:
    lat         (n_face, GLL_points) float64 -0.5417 -0.5359 ... -0.6836 -0.6903
    lon         (n_face, GLL_points) float64 5.445 5.46 5.483 ... 5.536 5.551

In [533]:
import pandas as pd

sr = pd.Series(test[0])
sr.index.name = 'n_face'
sr.to_xarray()

<xarray.DataArray ()>
array({2759, 31, 4471}, dtype=object)
Coordinates:
    n_face   int64 0

In [105]:
npts = 4
p_order = npts - 1
xinterior, w = roots_jacobi(p_order - 1,1,1) # returns interior GLL nodes from range -1 and 1
GLL_points = np.pad(xinterior, (1, 1), 'constant', constant_values=(-1, 1))

ref_coords = np.meshgrid(GLL_points, GLL_points[::-1])

In [106]:
node_x = grid_file.node_x.isel(n_node = node_connect)
node_y = grid_file.node_y.isel(n_node = node_connect)
node_z = grid_file.node_z.isel(n_node = node_connect)

In [107]:
t1 = (1 - ref_coords[0]) * (1 - ref_coords[1])
t2 = (1 + ref_coords[0]) * (1 - ref_coords[1])
t3 = (1 + ref_coords[0]) * (1 + ref_coords[1])
t4 = (1 - ref_coords[0]) * (1 + ref_coords[1])

In [120]:
# Returns Cartesian coordinates on the unit sphere: 

def map_cartesian(grid_file, ref_coords, GLL_points):
    
    # Define arrays for corners in x_2, y_2
    node_connect = grid_file.face_node_connectivity
    node_x = grid_file.node_x.isel(n_node = node_connect)
    node_y = grid_file.node_y.isel(n_node = node_connect)
    node_z = grid_file.node_z.isel(n_node = node_connect)
    
    t1 = (1 - ref_coords[0]) * (1 - ref_coords[1])
    t2 = (1 + ref_coords[0]) * (1 - ref_coords[1])
    t3 = (1 + ref_coords[0]) * (1 + ref_coords[1])
    t4 = (1 - ref_coords[0]) * (1 + ref_coords[1])
    
    r1 = (1/4) * (np.outer(node_x.isel(n_max_face_nodes = 0), t1.reshape(-1))
                   + np.outer(node_x.isel(n_max_face_nodes = 1), t2.reshape(-1)) 
                   + np.outer(node_x.isel(n_max_face_nodes = 2), t3.reshape(-1))
                   + np.outer(node_x.isel(n_max_face_nodes = 3), t4.reshape(-1)))
    
    r2 = (1/4) * (np.outer(node_y.isel(n_max_face_nodes = 0), t1.reshape(-1))
                   + np.outer(node_y.isel(n_max_face_nodes = 1), t2.reshape(-1)) 
                   + np.outer(node_y.isel(n_max_face_nodes = 2), t3.reshape(-1))
                   + np.outer(node_y.isel(n_max_face_nodes = 3), t4.reshape(-1)))
    
    r3 = (1/4) * (np.outer(node_z.isel(n_max_face_nodes = 0), t1.reshape(-1))
                   + np.outer(node_z.isel(n_max_face_nodes = 1), t2.reshape(-1)) 
                   + np.outer(node_z.isel(n_max_face_nodes = 2), t3.reshape(-1))
                   + np.outer(node_z.isel(n_max_face_nodes = 3), t4.reshape(-1)))
    
    r_vector = xr.Dataset(data_vars = {'x_comp' : (["n_face", "GLL_points"], r1), 'y_comp' : (["n_face", "GLL_points"], r2), 'z_comp' : (["n_face", "GLL_points"], r3)},
                          coords = {'n_face' : node_x.coords['n_face'], 'GLL_points' : np.arange(len(GLL_points) ** 2)})
    
    r_norm = r_vector / np.sqrt((r_vector['x_comp'] ** 2) + (r_vector['y_comp'] ** 2) + (r_vector['z_comp'] ** 2))
    return(r_norm)

In [151]:
# Convert Cartesian coordinates on the unit sphere to lat/lon in radians: 

def map_spherical(input_array):
    
    # Note: lon returned from [-pi, pi], lat returned from [0, pi]
    lon_array = np.arctan2(input_array['y_comp'], input_array['x_comp'])
    lat_array = np.arccos(input_array['z_comp'])
    
    # Shift lon to [0, 2pi] and lat to [pi/2, -pi/2]:
    lat_shift = -1 * (lat_array - (np.pi / 2))
    lon_shift = np.where(lon_array < 0, lon_array + (2 * np.pi), lon_array)
    
    return(xr.Dataset(data_vars = {'lat' : (["n_face", "GLL_points"], lat_shift.data), 
                                   'lon' : (["n_face", "GLL_points"], lon_shift.data)},
                      coords = {'n_face' : input_array.coords['n_face'], 'GLL_points' : input_array.coords['GLL_points']}))


In [540]:
spherical_coords = map_spherical(map_cartesian(grid_file, ref_coords, GLL_points))

In [538]:
spherical_coords['shared_corners'] = sr.to_xarray()

In [560]:
spherical_coords.isel(n_face = list(sr.to_xarray()[0]))

TypeError: iteration over a 0-d array

In [568]:
spherical_coords.isel(n_face = 10)

<xarray.Dataset>
Dimensions:     (GLL_points: 16)
Coordinates:
    n_face      int64 10
  * GLL_points  (GLL_points) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15
Data variables:
    lat         (GLL_points) float64 -0.7158 -0.7177 -0.7205 ... -0.7728 -0.7744
    lon         (GLL_points) float64 6.021 6.036 6.059 ... 6.036 6.059 6.074